In [1]:
import nltk
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

import torchtext

from tensorboardX import SummaryWriter
from tqdm import tqdm_notebook as tqdm

CUDA = torch.cuda.is_available()

In [2]:
BATCH_SIZE = 32

# Долгий способ сделать всё ЗБС

In [24]:
text = torchtext.data.Field(lower=True, include_lengths=True, batch_first=True)
label = torchtext.data.Field(sequential=True)

In [25]:
train, test = torchtext.datasets.IMDB.splits(text, label)

In [14]:
# print information about the data
print('train.fields', train.fields)
print('len(train)', len(train))
print('vars(train[0])', vars(train[0]))


train.fields {'text': <torchtext.data.field.Field object at 0x7f64bc9505c0>, 'label': <torchtext.data.field.Field object at 0x7f64bc950588>}
len(train) 25000
vars(train[0]) {'label': ['pos'], 'text': ['this', 'musical', 'is', 'decidedly', 'mixed,', 'and', 'none', 'of', 'the', 'elements', 'really', 'fit', 'together,', 'but', 'it', 'somehow', 'manages', 'to', 'be', 'mostly', 'enjoyable.', 'the', 'plot', 'contains', 'some', 'of', 'the', 'elements', 'of', "wodehouse's", 'novel,', 'but', 'none', 'of', 'its', 'virtues,', 'though', 'he', 'co-wrote', 'the', 'script.', 'the', 'songs,', 'though', 'charming,', 'have', 'nothing', 'to', 'do', 'with', 'this', 'particular', 'film,', 'and', 'are', 'unusually', 'crudely', 'squeezed', 'into', 'the', 'plot,', 'even', 'by', 'pre-oklahoma', 'standards.', 'burns', 'and', 'allen', 'do', 'their', 'usual', 'shtick', 'quite', 'competently,', 'but', 'it', 'misses', 'the', 'tone', 'of', 'the', 'rest', 'of', 'the', 'film', 'by', 'about', 'forty', 'iq', 'points.<br

In [20]:
vectors = torchtext.vocab.GloVe(name='6B', dim=300, cache='/media/data/nlp/data/')

In [26]:
text.build_vocab(train, vectors=vectors)

In [27]:
label.build_vocab(train)

In [29]:
print('len(TEXT.vocab)', len(text.vocab))
print('TEXT.vocab.vectors.size()', text.vocab.vectors.size())


len(TEXT.vocab) 251639
TEXT.vocab.vectors.size() torch.Size([251639, 300])


In [31]:
train_iter, test_iter = torchtext.data.BucketIterator.splits(
    (train, test), batch_size=16, device=0
)

In [41]:
# print batch information
batch = next(iter(train_iter))
print(batch.text)
print(batch.label)

Variable containing:
 9.0000e+00  1.9000e+01  1.5250e+03  ...   3.2250e+03  1.9000e+01  1.4100e+02
 9.0000e+01  1.2800e+02  2.7200e+02  ...   8.0000e+01  5.4000e+01  1.6700e+02
 2.0700e+02  1.7900e+02  4.0600e+02  ...   5.4563e+04  1.5720e+03  2.0000e+02
                ...                   ⋱                   ...                
 1.0000e+00  1.0000e+00  1.0000e+00  ...   1.2000e+01  1.5000e+01  6.0770e+04
 1.0000e+00  1.0000e+00  1.0000e+00  ...   7.0000e+00  2.0000e+00  4.0000e+00
 1.0000e+00  1.0000e+00  1.0000e+00  ...   1.0093e+04  9.5111e+04  1.2900e+02
[torch.cuda.LongTensor of size 211x32 (GPU 0)]

Variable containing:
 2
 1
 1
 1
 1
 2
 2
 1
 2
 2
 2
 1
 2
 2
 1
 1
 2
 2
 2
 1
 2
 1
 2
 1
 1
 2
 2
 1
 1
 2
 1
 2
[torch.cuda.LongTensor of size 32 (GPU 0)]



# Короткий способ сделать всё збс

In [35]:
train_iter, test_iter = torchtext.datasets.IMDB.iters(batch_size=BATCH_SIZE)

In [36]:
# print batch information
batch = next(iter(train_iter))
print(batch.text)
print(batch.label)

Variable containing:
 5.1504e+04  1.9000e+01  1.9000e+01  ...   1.6000e+02  4.9000e+01  1.2300e+02
 1.1248e+04  2.4100e+02  6.1000e+02  ...   7.8700e+02  1.0540e+03  2.8100e+02
 9.1800e+02  2.4000e+01  7.0000e+00  ...   6.5870e+03  1.1570e+03  6.0000e+00
                ...                   ⋱                   ...                
 1.0000e+00  1.0000e+00  1.0000e+00  ...   3.4000e+01  1.3000e+01  1.7000e+01
 1.0000e+00  1.0000e+00  1.0000e+00  ...   1.2089e+04  2.3394e+04  1.5300e+02
 1.0000e+00  1.0000e+00  1.0000e+00  ...   1.0000e+00  1.0000e+00  9.1000e+02
[torch.cuda.LongTensor of size 238x32 (GPU 0)]

Variable containing:
 1
 2
 2
 2
 2
 1
 1
 2
 2
 1
 1
 1
 1
 1
 1
 2
 1
 1
 1
 1
 2
 1
 2
 1
 2
 2
 2
 2
 1
 2
 2
 1
[torch.cuda.LongTensor of size 32 (GPU 0)]



# Попробуем простые модели
* CNN
* LSTM

#### Потом модели посложнее
* biLSTM
* SRU
* RNN_CNN
* RCNN

#### Отдельно стоит выделить класс
* Char CNN
* Char CNN with spip gram

In [42]:
text = torchtext.data.Field(lower=True, include_lengths=True, batch_first=True)
label = torchtext.data.Field(sequential=True)

train, test = torchtext.datasets.IMDB.splits(text, label)

In [49]:
train[0].label

['pos']

In [50]:
train[0].text

['this',
 'musical',
 'is',
 'decidedly',
 'mixed,',
 'and',
 'none',
 'of',
 'the',
 'elements',
 'really',
 'fit',
 'together,',
 'but',
 'it',
 'somehow',
 'manages',
 'to',
 'be',
 'mostly',
 'enjoyable.',
 'the',
 'plot',
 'contains',
 'some',
 'of',
 'the',
 'elements',
 'of',
 "wodehouse's",
 'novel,',
 'but',
 'none',
 'of',
 'its',
 'virtues,',
 'though',
 'he',
 'co-wrote',
 'the',
 'script.',
 'the',
 'songs,',
 'though',
 'charming,',
 'have',
 'nothing',
 'to',
 'do',
 'with',
 'this',
 'particular',
 'film,',
 'and',
 'are',
 'unusually',
 'crudely',
 'squeezed',
 'into',
 'the',
 'plot,',
 'even',
 'by',
 'pre-oklahoma',
 'standards.',
 'burns',
 'and',
 'allen',
 'do',
 'their',
 'usual',
 'shtick',
 'quite',
 'competently,',
 'but',
 'it',
 'misses',
 'the',
 'tone',
 'of',
 'the',
 'rest',
 'of',
 'the',
 'film',
 'by',
 'about',
 'forty',
 'iq',
 'points.<br',
 '/><br',
 '/>there',
 'are',
 'a',
 'few',
 'high',
 'points.',
 'reginald',
 'gardiner',
 'does',
 'good',

In [55]:
dataset = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
train.

In [ ]:
embeddings = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)

# CNN

In [37]:
writer = SummaryWriter(comment='_model_CNN')

In [ ]:
class LSTMClassifier(nn.Module):
    
    def __init__(self, embeddings, hidden_dim=128, batch_size=BATCH_SIZE):
        super(LSTMClassifier, self).__init__()
        self.embeddings = embeddings
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        
        self.lstm = nn.LSTM(self.embeddings.dim)